In [80]:
from sqlalchemy import insert

In [3]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

USER = "postgres"
PASSWORD = "ycode"
HOST = "localhost"
PORT = "5432"
DB_NAME = "breif5_football_db"

# Encoder le mot de passe pour gérer les caractères spéciaux
password_encoded = quote_plus(PASSWORD)
user_encoded = quote_plus(USER)

DATABASE_URL = f"postgresql+psycopg2://{user_encoded}:{password_encoded}@{HOST}:{PORT}/{DB_NAME}"

engine = create_engine(DATABASE_URL, echo=True)

try:
    with engine.connect() as conn:
        print("✅ Connexion réussie à la base PostgreSQL !")
except Exception as e:
    print("❌ Erreur de connexion :", e)


2025-10-31 15:34:14,950 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-31 15:34:14,951 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-31 15:34:14,954 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-31 15:34:14,955 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-31 15:34:14,958 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-31 15:34:14,959 INFO sqlalchemy.engine.Engine [raw sql] {}
✅ Connexion réussie à la base PostgreSQL !


In [ ]:
from sqlalchemy import MetaData
metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables.keys())

## acceder aux tables 
competition = metadata.tables['competition']
saison = metadata.tables['saison']
equipe = metadata.tables['equipe']
joueur = metadata.tables['joueur']
resultatmatch = metadata.tables['resultatmatch']
statistiquejoueur_saison = metadata.tables['statistiquejoueur_saison']
matchsEquipe = metadata.tables['matchsEquipe']


** insert competition  et saison

In [ ]:
# competition = Table(
#     "competition", metadata,
#     Column("idcompetition", Integer, primary_key=True, autoincrement=True),
#     Column("nomcompetition", String(100), nullable=False)
# )
with engine.connect() as conn :
    stmt = insert(competition).values(nomcompetition = "Premier League")
    conn.execute(stmt)
    conn.commit()


In [ ]:
# saison = Table(
#     "saison", metadata,
#     Column("idsaison", Integer, primary_key=True, autoincrement=True),
#     Column("annee", String(20), nullable=False)
# )
with engine.connect() as conn :
    stmt = insert(saison).values(annee = "2024/2025")
    conn.execute(stmt)
    conn.commit()

** insert equipe 

In [5]:
import os 
import pandas as pd 
import numpy as np
from datetime import datetime

SILVER_PATH = "../../data/silver"

# recupere les equipes , nom et leur path 
teams = [d for d in os.listdir(SILVER_PATH) 
            if os.path.isdir(os.path.join(SILVER_PATH, d))]
dict_teams_match_path= {}
dict_teams_players_path= {}
for team_dir in teams:
    team_path = os.path.join(SILVER_PATH, team_dir)
    team_name = team_dir.replace('_', ' ')      
    
    dict_teams_match_path[team_name] = os.path.join(team_path ,f"{team_dir}_matches_clean.csv") 
    dict_teams_players_path[team_name] = os.path.join(team_path ,f"{team_dir}_players_clean.csv") 


In [ ]:
dict_teams_match_path
dict_teams_players_path

In [ ]:
# equipe = Table(
#     "equipe", metadata,
#     Column("idequipe", Integer, primary_key=True, autoincrement=True),
#     Column("nomequipe", String(100), nullable=False),
# )

for team, path in dict_teams_match_path.items():
    with engine.begin() as conn:
        stmt = insert(equipe).values(nomequipe=team)
        conn.execute(stmt)



In [7]:
from sqlalchemy import select

def get_select(table, res_col_name, cond_col_name, cond_value):
    res_col = table.c[res_col_name]    # référence à la colonne à sélectionner
    cond_col = table.c[cond_col_name]  # référence à la colonne condition
    with engine.connect() as conn:
        stmt = select(res_col).where(cond_col == cond_value)
        result = conn.execute(stmt).fetchone()
        if result:
            return result[0]
        else:
            print(f"{cond_value} n'existe pas dans la table")
            return None

# Exemple d'utilisation
res = get_select(equipe, "idequipe", "nomequipe", "Arsenal")
print(res)



2025-10-31 16:01:31,377 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-31 16:01:31,379 INFO sqlalchemy.engine.Engine SELECT equipe.idequipe 
FROM equipe 
WHERE equipe.nomequipe = %(nomequipe_1)s
2025-10-31 16:01:31,380 INFO sqlalchemy.engine.Engine [generated in 0.00310s] {'nomequipe_1': 'Arsenal'}
2025-10-31 16:01:31,386 INFO sqlalchemy.engine.Engine ROLLBACK
21


In [ ]:
# joueur = Table(
#     "joueur", metadata,
#     Column("idjoueur", Integer, primary_key=True, autoincrement=True),
#     Column("nomjoueur", String(100), nullable=False),
#     Column("position", String(10)),
#     Column("nationalite", String(50)),
#     Column("idequipe", Integer, ForeignKey("equipe.idequipe"))
# )


# for team, path in dict_teams_players_path.items():
#     df = pd.read_csv(path)
#     id_equipe = get_select(equipe, "idequipe", "nomequipe", team)
#     with engine.begin() as conn:
#         stmt = insert(joueur).values(nomjoueur=df["Player"] , position=df["Pos"] , nationalite=df["nation"] , idequipe = id_equipe  )
#         conn.execute(stmt)


for team, path in dict_teams_players_path.items():
    df = pd.read_csv(path)
    id_equipe = get_select(equipe, "idequipe", "nomequipe", team)
    # Préparer la liste de dictionnaires pour tous les joueurs
    joueurs_data = [
        {
            "nomjoueur": row["Player"],
            "position": row["Pos"],
            "nationalite": row["Nation"],
            "idequipe": id_equipe
        }
        for _, row in df.iterrows()
    ]
    # Insérer tous les joueurs d'un coup
    with engine.begin() as conn:
        conn.execute(insert(joueur), joueurs_data)




In [ ]:
# matchs = Table(
#     "match", metadata,
#     Column("idmatch", Integer, primary_key=True, autoincrement=True),
#     Column("date_match", Date()),
#     Column("score", String(10)),
#     Column("round" , String(50)) ,
#     Column("jour"  , String(50)) ,
#     Column("arbitre" , String(100)),
#     Column("idcompetition" , Integer , ForeignKey("competition.idcompetition")) , 
#     Column("idSaision" , Integer , ForeignKey("saison.idsaison"))
#     )

#   inchangeable 
# id_competition = get_select(competition, "idcompetition", "nomcompetition", "Premier League")
# id_saison = get_select(saison, "idsaison","annee" , "2024/2025")

# GOLD_PATH_match_unique = "../../data/gold/matches_unique.csv"



# df = pd.read_csv(GOLD_PATH_match_unique)
# df.head(10)
# match_data = [
#     {
#         'eq1': row["eq1"],
#         'eq2': row["eq2"],
#         'date_match': row["date_match"],
#         'heure': row["heure"],
#         'round': row["round"],
#         'jour': row["jour"],
#         'arbitre': row["arbitre"],
#         'idcompetition': 1,
#         'idSaison': 1
#     }
#     for _, row in df.iterrows()
# ]
# with engine.begin() as conn:
#  conn.execute(insert(matchsEquipe), match_data)



insert resultat match

In [ ]:
# resultatmatch = Table(
#     "resultatmatch", metadata,
#     Column("idresultat", Integer, primary_key=True, autoincrement=True),
#     Column("idmatch", Integer, ForeignKey("matchsEquipe.idmatch")),
#     Column("idequipe", Integer, ForeignKey("equipe.idequipe")),
#     Column("venue", Enum("Home", "Away", "Neutral", name="venue_enum")),
#     Column("butsmarques", Integer),
#     Column("butsconcedes", Integer),
#     Column("resultat", Enum("Victoire", "Défaite", "Nul", name="resultat_enum")),
#     Column("xG", Numeric(3, 1)),
#     Column("xGA", Numeric(3, 1)),
#     Column("possession", Numeric(4, 1)),
#     Column("capitaine", String(100)),
#     Column("formation", String(20)),
# )


import pandas as pd
from sqlalchemy import select, insert, or_

# 1️⃣ Charger les matchs déjà présents en base
with engine.connect() as conn:
    matchs_db = pd.read_sql(select(matchsEquipe), conn)

# 2️⃣ Charger le CSV des résultats nettoyés
GOLD_PATH_RESULTAT = "../../data/gold/matches_resultat_unique.csv"
df_resultat = pd.read_csv(GOLD_PATH_RESULTAT)

# 3️⃣ Préparer les données à insérer
insert_data = []

for _, row in df_resultat.iterrows():
    date_df = row['match'].split('_')[0] if '_vs_' in row['match'] else row['match']

   
    eq1 = row['equipe']
    eq2 = row['match'].split('_vs_')[1] if '_vs_' in row['match'] else None
    print(f"{date_df} , {eq1} , {eq2} ")

 
    # Chercher le match correspondant dans la table matchsEquipe
    match_row = matchs_db[
        ((matchs_db['date_match'] == date_df) &
         (matchs_db['eq1'] == eq1) &
         (matchs_db['eq2'] == eq2))
        |
        ((matchs_db['date_match'] == date_df) &
         (matchs_db['eq1'] == eq2) &
         (matchs_db['eq2'] == eq1))
    ]

    if match_row.empty:
        continue  # match non trouvé → on saute la ligne

    idmatch = int(match_row.iloc[0]['idmatch'])

    # Récupérer l'id de l'équipe (en utilisant ta fonction utilitaire)
    id_equipe = get_select(equipe, "idequipe", "nomequipe", eq1)

    # Ajouter à la liste d’insertion
    insert_data.append({
        'idmatch': idmatch,
        'idequipe': id_equipe,
        'venue': row['venue'],
        'butsmarques': row['butsmarques'],
        'butsconcedes': row['butsconcedes'],
        'resultat': row['resultat'],
        'xG': row['XG'],
        'xGA': row['XGA'],
        'possession': row['possession'],
        'capitaine': row['capitaine'],
        'formation': row['formation']
    })

# 4️⃣ Insertion dans la table resultatmatch
if insert_data:
    with engine.begin() as conn:
        conn.execute(insert(resultatmatch), insert_data)
    print(f"✅ {len(insert_data)} lignes insérées dans resultatmatch.")
else:
    print("⚠️ Aucune donnée insérée — aucun match correspondant trouvé.")




In [ ]:
matchs_db

In [ ]:
import pandas as pd
from sqlalchemy import select, insert
from sqlalchemy.orm import Session

# 1️⃣ Charger les matchs déjà présents en base
with engine.connect() as conn:
    matchs_db = pd.read_sql(select(matchsEquipe), conn)

# Normalisation pour éviter les problèmes de correspondance
matchs_db['eq1'] = matchs_db['eq1'].str.strip().str.lower()
matchs_db['eq2'] = matchs_db['eq2'].str.strip().str.lower()
matchs_db['date_match'] = pd.to_datetime(matchs_db['date_match']).dt.strftime('%Y-%m-%d')

# 2️⃣ Charger le CSV des résultats nettoyés
GOLD_PATH_RESULTAT = "../../data/gold/matches_resultat_unique.csv"
df_resultat = pd.read_csv(GOLD_PATH_RESULTAT)

# Normalisation des données du CSV
df_resultat['match'] = df_resultat['match'].str.strip()
df_resultat['equipe'] = df_resultat['equipe'].str.strip().str.lower()
df_resultat['venue'] = df_resultat['venue'].str.strip()
df_resultat['resultat'] = df_resultat['resultat'].str.strip()

# 3️⃣ Préparer les données à insérer
insert_data = []

for _, row in df_resultat.iterrows():
    # Extraire date et adversaire du champ 'match'
    date_df = row['match'].split('_')[0].strip()
    eq1 = row['equipe']
    eq2 = row['match'].split('_vs_')[1].strip().lower() if '_vs_' in row['match'] else None

    # Chercher le match correspondant dans la table matchsEquipe
    match_row = matchs_db[
        ((matchs_db['date_match'] == date_df) &
         (matchs_db['eq1'] == eq1) &
         (matchs_db['eq2'] == eq2)) |
        ((matchs_db['date_match'] == date_df) &
         (matchs_db['eq1'] == eq2) &
         (matchs_db['eq2'] == eq1))
    ]

    if match_row.empty:
        print(f"⚠️ Match non trouvé: {date_df} {eq1} vs {eq2}")
        continue

    idmatch = int(match_row.iloc[0]['idmatch'])

    # Récupérer l'id de l'équipe
    id_equipe = get_select(equipe, "idequipe", "nomequipe", eq1.strip().title())
    print(f"_____________________{eq1} , {id_equipe}_______")

    # Ajouter à la liste d’insertion
    insert_data.append({
        'idmatch': idmatch,
        'idequipe': id_equipe,
        'venue': row['venue'],
        'butsmarques': row['butsmarques'],
        'butsconcedes': row['butsconcedes'],
        'resultat': row['resultat'],
        'xG': row['XG'],
        'xGA': row['XGA'],
        'possession': row['possession'],
        'capitaine': row['capitaine'],
        'formation': row['formation']
    })
insert_data
# 4️⃣ Insertion dans la table resultatmatch
if insert_data:
    with engine.begin() as conn:
        conn.execute(insert(resultatmatch), insert_data)
    print(f"✅ {len(insert_data)} lignes insérées dans resultatmatch.")
else:
    print("⚠️ Aucune donnée insérée — aucun match correspondant trouvé.")


In [15]:
import pandas as pd
from sqlalchemy import select, insert
from sqlalchemy.orm import Session


# statistiquejoueur_saison = Table(
#     "statistiquejoueur_saison", metadata,
#     Column("idstats", Integer, primary_key=True, autoincrement=True),
#     Column("idjoueur", Integer, ForeignKey("joueur.idjoueur")),
#     Column("matchs_joues", Integer),
#     Column("buts", Integer),
#     Column("passes_decisives", Integer),
#     Column("cartons_jaunes", Integer),
#     Column("cartons_rouges", Integer)
# )


for team, path in dict_teams_players_path.items():
    df = pd.read_csv(path)
    
   
    # Préparer la liste de dictionnaires pour tous les joueurs
    joueurs_stat_data = [
        {
            "idjoueur": get_select(joueur, "idjoueur", "nomjoueur", row["Player"]),
            "matchs_joues": row["MP"],
            "buts": row["Gls"],
            "passes_decisives" : row["Ast"] , 
            "buts_passes" : row["G+A"],
            "cartons_jaunes" : row["CrdY"] , 
            "cartons_rouges" : row["CrdR"]               
        }
        for _, row in df.iterrows()
    ]
    # Insérer tous les joueurs d'un coup
    with engine.begin() as conn:
        conn.execute(insert(statistiquejoueur_saison), joueurs_stat_data)

joueurs_stat_data
df.head()

2025-10-31 16:19:35,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-31 16:19:35,539 INFO sqlalchemy.engine.Engine SELECT joueur.idjoueur 
FROM joueur 
WHERE joueur.nomjoueur = %(nomjoueur_1)s
2025-10-31 16:19:35,540 INFO sqlalchemy.engine.Engine [cached since 1076s ago] {'nomjoueur_1': 'David Raya'}
2025-10-31 16:19:35,544 INFO sqlalchemy.engine.Engine ROLLBACK
2025-10-31 16:19:35,546 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-31 16:19:35,547 INFO sqlalchemy.engine.Engine SELECT joueur.idjoueur 
FROM joueur 
WHERE joueur.nomjoueur = %(nomjoueur_1)s
2025-10-31 16:19:35,548 INFO sqlalchemy.engine.Engine [cached since 1076s ago] {'nomjoueur_1': 'William Saliba'}
2025-10-31 16:19:35,550 INFO sqlalchemy.engine.Engine ROLLBACK
2025-10-31 16:19:35,553 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-31 16:19:35,554 INFO sqlalchemy.engine.Engine SELECT joueur.idjoueur 
FROM joueur 
WHERE joueur.nomjoueur = %(nomjoueur_1)s
2025-10-31 16:19:35,555 INFO sqlalchemy.eng

,Player,Team,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,season,league,processing_date
0,Rayan Aït-Nouri,Wolves,DF,23.0,37,37,0.0,34,4.0,7.0,11.0,4.0,0.0,0.0,7.0,1.0,2.7,2024-2025,Premier League,2025-10-28
1,João Gomes,Wolves,MF,23.0,36,35,0.0,33,3.0,1.0,4.0,3.0,0.0,0.0,11.0,1.0,2.9,2024-2025,Premier League,2025-10-28
2,Nélson Semedo,Wolves,DF,30.0,34,32,0.0,32,0.0,4.0,4.0,0.0,0.0,0.0,8.0,0.0,0.9,2024-2025,Premier League,2025-10-28
3,André,Wolves,MF,23.0,33,31,0.0,27,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.4,2024-2025,Premier League,2025-10-28
4,Toti Gomes,Wolves,DF,25.0,31,30,0.0,29,0.0,1.0,1.0,0.0,0.0,0.0,7.0,0.0,0.7,2024-2025,Premier League,2025-10-28
